###  Libraries

In [ ]:
import numpy as np

from Related_Codes.DCNAutoencoder import DCNAutoencoder, DCNCDAutoencoder
from Related_Codes.DECAutoencoder import DECAutoencoder, DECCDAutoencoder
from Related_Codes.IDECAutoencoder import IDECAutoencoder, IDECCDAutoencoder
from General_Functions import General_Functions

from sklearn.decomposition import PCA, TruncatedSVD, NMF
from sklearn.manifold import LocallyLinearEmbedding, SpectralEmbedding, Isomap, TSNE

import torch

import os
import sys

from Datasets.Datasets_Functions import *
from Visualization import Visualization

### Global Variables

In [ ]:
path_to_module = './'
sys.path.append(path_to_module)
os.environ['OMP_NUM_THREADS'] = '6'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)

### Load Dataset

In [ ]:
dataset_name = "emnist_balanced_letters_A_J" # The available datasets are: emnist_balanced_digits, emnist_mnist, emnist_balanced_letters_A_J, emnist_balanced_letters_K_T, emnist_balanced_letters_U_Z, har, pendigits, waveform_v1, synthetic
module_name = "Datasets.Datasets"
batch_size = 256
n_clusters = 10
dataloader, input_dim, data_np, labels = function_get_dataset(dataset_name, module_name, batch_size, n_clusters)
print('Data_Shape is:', input_dim)
print('Batches Number is:', len(dataloader))
print(np.unique(labels))

### Visualization

In [ ]:
visualization = Visualization()

### Parameters

In [ ]:
# Set latent dimension and negative slope
latent_dim = 5
negative_slope = 0

# Set pre-training epochs and learning rate
n_pret_epochs = 1000
pret_lr = 5e-4

# Set lamda, beta, training epochs and learning rate
lamda = 1
beta = 1e-5
n_epochs = 100
lr = 5e-4

# Set use_pretrain and is_mlp_ae
use_pretrain = True
is_mlp_ae = False

### Create Autoencoder Model

In [ ]:
if(is_mlp_ae):
    dcn_autoencoder = DCNAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, negative_slope=negative_slope)
else:  
    input_dim = 1
    dcn_autoencoder = DCNCDAutoencoder(device=device, n_clusters=n_clusters, input_dim=input_dim, latent_dim=latent_dim, negative_slope=negative_slope) 
dcn_autoencoder.set_general_training_variables(dataloader=dataloader, batch_size=batch_size)
dcn_autoencoder.set_pretraining_variables(n_pret_epochs=n_pret_epochs, pret_lr=pret_lr)
dcn_autoencoder.set_training_variables(n_epochs=n_epochs, lr=lr, lamda=lamda, beta=beta )
dcn_autoencoder.set_path_variables(path_to_module=path_to_module, dataset_name=dataset_name)
dcn_autoencoder.set_path()
dcn_autoencoder = dcn_autoencoder.to(device)

### Pretrain Autoencoder

In [ ]:
if(use_pretrain):
    dcn_autoencoder.pretrain_autoencoder()
    dcn_autoencoder.save_pretrained_weights()
else:
    model_save_path = dcn_autoencoder.data_dir_path + '/Weigths/autoencoder_weights.pth'
    dcn_autoencoder.load_state_dict(torch.load(model_save_path))

### Train Autoencoder

In [ ]:
dcn_autoencoder.train_autoencoder()

### Save Clustering Results

In [ ]:
General_Functions().save_excel(dcn_autoencoder.data_dir_path, dcn_autoencoder.df_eval)

### Plot Clustering Results

In [ ]:
cluster_centers = dcn_autoencoder.get_cluster_centers()
_, dcn_autoencoder_reduced_data, labels = dcn_autoencoder.get_latent_data()
visualization.plot_tsne(dcn_autoencoder_reduced_data, labels, labels, cluster_centers, dcn_autoencoder.data_dir_path)